<a href="https://colab.research.google.com/github/gabrielhozana/Export-Data-to-Training-Pipelines-TFDS-/blob/main/Export_Data_to_Training_Pipelines_(TFDS).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# https://www.kaggle.com/shivam2503/diamonds
!wget https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv

--2021-07-06 13:31:45--  https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3246501 (3.1M) [text/plain]
Saving to: ‘diamonds.csv’

diamonds.csv        100%[===================>]   3.10M  --.-KB/s    in 0.1s    

2021-07-06 13:31:46 (24.1 MB/s) - ‘diamonds.csv’ saved [3246501/3246501]



In [ ]:
import pandas as pd
import numpy as np
 
import tensorflow as tf
 
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

# url= 'https://raw.githubusercontent.com/natashayulian/diamond_dataset/master/diamonds.csv'
# df = pd.read_csv(url)
df = pd.read_csv("diamonds.csv")

In [ ]:
pd.concat([df.head(),df.tail()])

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
53935,53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53936,53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53937,53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53938,53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74
53939,53940,0.75,Ideal,D,SI2,62.2,55.0,2757,5.83,5.87,3.64


In [ ]:
# Dalam kasus ini, saya membagi dataset bila harga berlian kurang dari sama dengan 1000 maka target = 0, selebihnya maka 1

df['target'] = np.where(df['price'] <= 1000, 0, 1)

In [ ]:
df.head()

,Unnamed: 0,carat,cut,color,clarity,depth,table,price,x,y,z,target
0,1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43,0
1,2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31,0
2,3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31,0
3,4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63,0
4,5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75,0


In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

34521 train examples
8631 validation examples
10788 test examples


Bungkus dataframe dengan tf.data agar memungkinkan untuk menggunakan feature column sebagai jembatan dari dataframe menjadi fitur yang digunakan untuk train model. Jika file CSV nya sangat besar, kita harus menggunakan tf.data untuk membaca file tersebut dari harddisk.

In [ ]:
# Cara untuk membuat dataset tf.data dari pandas dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = df.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
  return ds
batch_size = 10 #bath ukuran kecil untuk demonstrasi
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

Beberapa jenis dari feature column yang dapat menjadi masukan TensorFlow


Sumber : https://www.tensorflow.org/tutorials/structured_data/feature_columns#choose_which_columns_to_use

Numeric column: Numeric column merupakan jenis feature column yang paling sederhana dan digunakan untuk merepresentasikan sebuah fitur dengan value yang apa adanya. Masukkan ke TensorFlow harus berupa angka dan numeric column sudah merupakan angka sehingga tidak perlu diubah lagi.

In [ ]:
example_batch = next(iter(train_ds))[0]

def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

# Lanjutkan dengan kode di modul bagian "jenis dari feature column yang dapat menjadi masukan TensorFlow"

In [ ]:
#numeric column
carat = feature_column.numeric_column('carat')
demo(carat)

[[0.5 ]
 [1.21]
 [1.2 ]
 [1.65]
 [0.63]
 [0.8 ]
 [1.51]
 [1.01]
 [0.32]
 [0.42]]


Bucketized column: Terkadang kita memiliki dataset numerik yang memiliki value beragam dengan range cukup jauh. Daripada memasukkan setiap value ke dalam numeric column, kita dapat menggunakan bucketized column untuk membagi value-value tersebut.

In [ ]:
#bucketized column
carat = feature_column.numeric_column('carat')
carat_buckets = feature_column.bucketized_column(carat, boundaries=[1, 2])
demo(carat_buckets)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 0. 0.]]


Categorical column: Kita tidak dapat memasukkan value String ke dalam TensorFlow. Oleh karena itu, categorical column digunakan untuk mengubah value String pada dataset (pada dataset ini contohnya cut dan clarity) menjadi angka.

In [ ]:
#categorical
color_type = feature_column.categorical_column_with_vocabulary_list(
    'color', ['E', 'I','J','D','H', 'G','F'])
     
color_type_one_hot = feature_column.indicator_column(color_type)
demo(color_type_one_hot)

[[0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]]


Embedding column: Jika kita memiliki banyak value dalam satu categorical column, lalu jenis value tersebut dapat bertambah seiring dengan berjalannya waktu, data tersebut menjadi tidak cocok jika hanya direpresentasikan dengan nilai 0 atau 1 seperti categorical column. Embedding column merepresentasikan satu categorical column dengan nilai yang beragam.

Key point: hasil embedding column menjadi maksimal jika sebuah categorical column memiliki banyak jenis value.

In [ ]:
#embedding
clarity = feature_column.categorical_column_with_vocabulary_list(
    'clarity', df.clarity.unique())
clarity_embedding = feature_column.embedding_column(clarity, dimension=6)
demo(clarity_embedding)

[[-0.7829155   0.03330744 -0.6025387  -0.14399634 -0.01922497  0.5234347 ]
 [ 0.46725932  0.4740839   0.51001245 -0.5959371   0.1775913   0.21511836]
 [-0.7414734  -0.0485617  -0.69867444  0.00184668  0.27701038  0.22357686]
 [ 0.46725932  0.4740839   0.51001245 -0.5959371   0.1775913   0.21511836]
 [-0.7829155   0.03330744 -0.6025387  -0.14399634 -0.01922497  0.5234347 ]
 [ 0.48188254  0.2606173  -0.33588216 -0.3203899   0.32082552  0.06529851]
 [-0.4330432  -0.45836148 -0.18844193 -0.00632132  0.06096065 -0.40578508]
 [-0.4330432  -0.45836148 -0.18844193 -0.00632132  0.06096065 -0.40578508]
 [ 0.19533172  0.21611315  0.5693507   0.24837926 -0.24299984 -0.24976754]
 [-0.7829155   0.03330744 -0.6025387  -0.14399634 -0.01922497  0.5234347 ]]


Hashed feature column: Hashed feature column merupakan cara alternatif untuk merepresentasikan categorical column yang memiliki banyak jenis value. Kita dapat menentukan jumlah hash_buckets jauh lebih sedikit dari jumlah kategori yang sebenarnya untuk menghemat tempat.

Key point: kerugian dari teknik ini adalah dapat terjadi collision yang mana kategori berbeda dipetakan pada bucket yang sama.

In [ ]:
#hashed feature
clarity_hashed = feature_column.categorical_column_with_hash_bucket('clarity', hash_bucket_size=5)
demo(feature_column.indicator_column(clarity_hashed))

[[0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


Crossed feature column: Crossed feature column menggabungkan banyak feature column menjadi satu feature column. Crossed feature column membuat column feature baru yang memungkinkan model untuk mempelajari weight berbeda untuk setiap kombinasi dari column feature.

Catatan: crossed feature column tidak membuat table dari seluruh kemungkinan kombinasi feature column karena ukurannya bisa menjadi sangat besar. Sebagai gantinya, crossed feature column didukung oleh hashed feature column sehingga dapat mendefinisikan ukuran table tersebut.

In [ ]:
#cross feature
#data yang di cross harus berupa string, categorical, atau bucketized
crossed_feature = feature_column.crossed_column([carat_buckets, color_type],
                                                hash_bucket_size=10)
demo(feature_column.indicator_column(crossed_feature))

[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


Kemudian buat feature layer sebagai input ke dalam model tf.Keras

In [ ]:
#membuat feature layer
#untuk feature columns dapat dicek disini https://www.tensorflow.org/tutorials/structured_data/feature_columns#create_a_feature_layer
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
#create, compile, and train the model
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dropout(.1),
  layers.Dense(1)
])
 
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])
 
model.fit(train_ds,
          validation_data=val_ds,